# Conway's Game of Life

Authors: Edwin Weill & Brad Green  
Due Date: November 29th  

This iPython notebook serves as the project code for the final project in MATH 8650 (Data Structures).  

__Project Description__: Conway's game of life is a cellular automaton devised by John Horton Conway in 1970.
The "game" is structured so that the evolution is only based on the initial state, meaning no user input is needed.  Initial configurations can be created randomly or by creating known patterns with particular properties.  

A set of rules is derived that mimics growth of a colony of some biological organisms.  In most cases, the "game" is played on a two-dimensional grid which contains "dead" and "living" cells.  The following are a small subset of the rules that govern the evolution of the "game".  

- __Reproduction__: If a "dead" cell is surrounded by exactly 3 "living" cells, it become a "living" cell
- __Underpopulation__: If a "living" cell is surrounded by fewer than two "living" cells, it dies.
- __Overpopulation__: If a "living" cell is surrounded by more than three "living" cells, it dies.
- __Stasis__: If a "living" cell is surrounded by two or three "living" cells, it survives.

### Import necessary libraries

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from random import randint
from copy import deepcopy

%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### Set up display for animation

In [2]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

### Conway Game of Life Grid Class

In [116]:
class ConwayGOLGrid():
    """
    Represents a grid in the Conway's Game of Life problem where
    each of the cells contained in the grid may be either alive or
    dead for any given state.
    """
    def __init__(self, width=100, height=100, startCells=[], optimized=True):
        """
        Initializes a Grid as a 2D list and comprised of Cells.
        
        Parameters: Width, Height as size of the board
                    startCells as a list of cells to start as alive.
                        If startCells is empty, cells will spawn as alive
                        at a rate of 30%.
                        startCells should be a list of coordinates, (x,y)
                    optimized determines whether or not to use data structures
                        to improve overall run-time.
        """
        
        self.width, self.height = width, height
        self.__optimized = optimized
        self.cells = []
        self.__living = set()
        
        
        for x in range(self.width):
            # Create a new list for 2D structure
            self.cells.append([])
            
            for y in range(self.height):
                # If no startCells provided, randomly init as alive
                if len(startCells) == 0 and randint(0,100) < 30:
                    self.cells[x].append(ConwayGOLCell(x, y, True))
                    self.__living.add((x,y))
                    
                else:
                    self.cells[x].append(ConwayGOLCell(x,y))
                    
        # Give life to all cells in the startCells list
        for cell in startCells:
            self.cells[cell[0]][cell[1]].spawn()
            self.__living.add((cell))

        
        
    def update(self):
        """
        Updates the current state of the game using the standard
        Game of Life rules.
        
        Parameters: None
        Returns: True if there are remaining alive cells.
                 False otherwise.
        """
        alive = False
        
        if not self.__optimized:
            # Deep copy the list to make sure the entire board updates correctly
            tempGrid = deepcopy(self.cells)

            # For every cell, check the neighbors.
            for x in range(self.width):
                for y in range(self.height):
                    neighbors = self.cells[x][y].num_neighbors(self)
                    
                    # Living cells stay alive with 2 or 3 neighbors, else die
                    if self.cells[x][y].is_alive():
                        if neighbors != 2 and neighbors != 3:
                            tempGrid[x][y].die()
                        else:
                            alive = True
                            
                    # Non living cells come alive with 3 neighbors
                    else:
                        if neighbors == 3:
                            tempGrid[x][y].spawn()
                            alive = True
                        
            # Deep copy the tempGrid to prevent losing the reference when function is over
            self.cells = deepcopy(tempGrid)
            
        else:
            count = [[0 for y in range(self.height)] for x in range(self.width)]
            to_check = set()
            
            # For each cell that is alive...
            for cell in self.__living:
                x, y = cell
                to_check.add(cell)
                
                # Grab all of its neighbors
                for neighbor in self.cells[x][y].neighbors:
                    n_x, n_y = neighbor
                    # If the neighbors are valid
                    if (    n_x >= 0 and n_y >= 0 and
                            n_x < self.width and n_y < self.height):
                        # Then increment their count and add them to a set
                        count[n_x][n_y] += 1
                        to_check.add(neighbor)
                            
               
            # Then start over living.
            self.__living = set()
            
            # Above, we add 1 to the count each time a cell is touched by an alive cell.
            # So we know count contains the number of alive neighbors any given cell has.
            # We use this to quickly check the rules of life and add cells to living array as needed.
            for cell in to_check:
                x, y = cell
                
                if self.cells[x][y].is_alive():
                    if count[x][y] != 2 and count[x][y] != 3:
                        self.cells[x][y].die()
                    else:
                        self.__living.add(cell)
                        alive = True
                else:
                    if count[x][y] == 3:
                        self.cells[x][y].spawn()
                        self.__living.add(cell)
                        alive = True
        
        return alive
    
    def print_grid(self):
        """
        Prints the current state of the board using text.
        
        Parameters: None
        Return: None
        """
        
        for y in range(self.height):
            for x in range(self.width):
                if self.cells[x][y].is_alive():
                    print "X  " ,
                else:
                    print ".  " ,
            print "\n"
            
        print "\n\n"
        
        

### Conway Game of Life Cell Class

In [103]:
class ConwayGOLCell():
    """
    Represents a cell in the Conway's Game of Life problem where
    a cell can either be alive or dead and the next state of the
    cell is based on the states of the immediate (8) neighbors.
    """
    def __init__(self, x, y, alive=False):
        """
        Create information for the given cell including the x and
        y coordinates of the cell, whether it is currently alive
        or dead, it's neighbors, and its current color.
        
        Parameters: x, y give the coordinate of the cell in grid
                    alive gives the current state of the cell
        Returns: None
        """
        self.x, self.y = x, y
        self.alive = alive
        self.neighbors = [(x-1,y-1), (x, y-1), (x+1, y-1),
                          (x-1,y  ),           (x+1, y  ),
                          (x-1,y+1), (x, y+1), (x+1, y+1)]
        self.color = (255,255,255)
        
    def spawn(self):
        """
        Changes the state of a cell from dead to alive.  Assumes
        that the cell is dead to be changed to alive (no need to
        modify if already alive).
        
        Parameters: None
        Returns: None
        """
        assert self.alive==False
        self.alive = True
        
    def die(self):
        """
        Changes the stat of a cell from alive to dead.  Assumes
        that the cell is alive to be changed to dead (no need to 
        modify if already dead).
        
        Parameters: None
        Returns: None
        """
        assert self.alive==True
        self.alive = False
        
    def is_alive(self):
        """
        Returns status of a cell.
        
        Parameters: None
        Returns: True if cell is alive.
        """
        return self.alive
        
    def num_neighbors(self, grid):
        """
        Returns the number of neighbors of a cell.
        
        Parameters: grid is the ConwayGOLGrid object that contains all cells
        Returns: Number of alive neighbors
        """
        
        num_neighbors = 0
        
        for cell in self.neighbors:
            x,y = cell
            
            if (    x >= 0 and x < grid.width and
                    y >= 0 and y < grid.height and
                    grid.cells[x][y].is_alive()):
                num_neighbors += 1
                    
        return num_neighbors

### Animate Grid

In [121]:
test_game = ConwayGOLGrid(10,10, optimized=True)

test_game.print_grid()
count = 0

while test_game.update():
    if count % 10 == 0:
        print "Iteration ", count
        test_game.print_grid()
        
    if count > 100:
        break
    count += 1
        
print "Finsihed after ", count, "iterations"

.   .   X   .   .   X   .   .   .   X   

.   .   X   .   .   .   .   .   .   .   

.   .   X   X   X   X   .   X   .   .   

X   X   X   .   X   .   .   X   X   .   

.   X   .   .   X   .   X   .   .   .   

X   .   .   X   X   X   .   .   X   .   

X   .   .   .   X   .   .   X   .   .   

.   X   .   X   .   .   .   .   X   .   

.   .   X   .   .   .   .   .   .   .   

.   X   .   .   X   .   .   .   X   .   




Iteration  0
.   .   .   .   .   .   .   .   .   .   

.   X   X   .   .   X   X   .   .   .   

.   .   .   .   X   X   X   X   X   .   

X   .   .   .   .   .   .   X   X   .   

.   .   .   .   .   .   X   .   X   .   

X   X   .   X   .   .   X   X   .   .   

X   X   X   .   .   X   .   X   X   .   

.   X   X   X   .   .   .   .   .   .   

.   X   X   X   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   




Iteration  10
.   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   

.